In [1]:
import pickle
import numpy as np

In [2]:
with open('documents.bin', 'rb') as f_in:
    results = pickle.load(f_in)

print("# de documentos:", len(results))
results[0]

# de documentos: 15


{'id': '2a3d5c52',
 'url': 'https://www.metrocuadrado.com/noticias/actualidad/baja-de-tasas-de-interes-facilita-la-compra-de-vivienda-en-2024-4799',
 'title': 'baja de tasas de interes facilita la compra de vivienda en 2024',
 'content': 'la compra de vivienda para muchos colombianos se ha visto afectada por diferentes factores economicos como la inflacion, devaluacion de la moneda, entre otros. frente a este panorama, muchos sectores, como el financiero, presentan medidas para la reactivacion de la economia.\nel banco de la republica viene haciendo reduccion de tasas de interes desde febrero del 2024, llegando en julio a 11.25%, y logrando a los mismos valores de octubre de 2022 cuando inicio un ascenso considerable de estas tasas.\nestas medidas, se hacen efectivas y positivas para los compradores de vivienda porque desde los primeros dias de julio, 10 entidades bancarias se han sumado y han bajado las tasas de interes para:\nlas 10 entidades bancarias que han entrado en la denominad

In [3]:
def describe_docs(num_caracteres: list):
    print(
        f"min: {round(np.min(num_caracteres))}", 
        f"mean: {round(np.mean(num_caracteres))}",
        f"median: {round(np.median(num_caracteres))}",
        f"max: {round(np.max(num_caracteres))}"
    )

In [4]:
# solo el contenido
num_caracteres = [len(doc["content"]) for doc in results]
describe_docs(num_caracteres)

min: 3223 mean: 7575 median: 6807 max: 15569


In [5]:
# titulo
num_caracteres = [len(doc["title"]) for doc in results]
describe_docs(num_caracteres)

min: 34 mean: 56 median: 59 max: 93


In [6]:
# titulo + contenido
num_caracteres = [len(doc["title"]+"\n"+doc["content"]) for doc in results]
describe_docs(num_caracteres)

min: 3287 mean: 7632 median: 6856 max: 15618


De acuerdo con los modelos en la documentación [sentence_transformer/pretrained_models](https://www.sbert.net/docs/sentence_transformer/pretrained_models.html) y en el [leaderboard](https://huggingface.co/spaces/mteb/leaderboard) (los que mejor performance promedio tienen y el uso de memoria en GB no es muy elevado), vamos a probar los siguientes modelos:


* [multilingual-e5-large-instruct](https://huggingface.co/intfloat/multilingual-e5-large-instruct)
    + Max Tokens: 514
    + Uso de memoria GB: 2.09
    + Dimension embeddings: 1024
    + https://www.pinecone.io/learn/the-practitioners-guide-to-e5/


* [multilingual-e5-small](https://huggingface.co/intfloat/multilingual-e5-small)
    + Max Tokens: 512
    + Uso de memoria GB: 0.44
    + Dimension embeddings: 384

* [paraphrase-multilingual-mpnet-base-v2](https://huggingface.co/sentence-transformers/paraphrase-multilingual-mpnet-base-v2)
    + Max Tokens: 514
    + Uso de memoria GB: 1.04
    + Dimension embeddings: 768


Teniendo en cuenta la cantidad máxima de tokens de cada modelo y que los documentos que tenemos evidentemente superan esta cantidad de tokens, entonces realizaremos un preprocesamiento de los doumentos, dividiendolos en chunks.

### Preprocesamiento

In [8]:
from langchain_text_splitters import CharacterTextSplitter

In [9]:
documents = [doc["content"] for doc in results]
metadatas = [
    {"id": doc["id"], "title": doc["title"]} 
    for doc in results
]

In [10]:
text_splitter = CharacterTextSplitter(
    separator=" ",
    chunk_size=400,
    length_function=len,
)
texts = text_splitter.create_documents(documents, metadatas=metadatas)
len(texts)
print(texts[0])

page_content='la compra de vivienda para muchos colombianos se ha visto afectada por diferentes factores economicos como la inflacion, devaluacion de la moneda, entre otros. frente a este panorama, muchos sectores, como el financiero, presentan medidas para la reactivacion de la economia.
el banco de la republica viene haciendo reduccion de tasas de interes desde febrero del 2024, llegando en julio a 11.25%, y' metadata={'id': '2a3d5c52', 'title': 'baja de tasas de interes facilita la compra de vivienda en 2024'}


In [11]:
for i in range(len(texts)):
    texts[i].page_content = texts[i].metadata["title"] +"\n"+ texts[i].page_content

In [12]:
print(texts[0])

page_content='baja de tasas de interes facilita la compra de vivienda en 2024
la compra de vivienda para muchos colombianos se ha visto afectada por diferentes factores economicos como la inflacion, devaluacion de la moneda, entre otros. frente a este panorama, muchos sectores, como el financiero, presentan medidas para la reactivacion de la economia.
el banco de la republica viene haciendo reduccion de tasas de interes desde febrero del 2024, llegando en julio a 11.25%, y' metadata={'id': '2a3d5c52', 'title': 'baja de tasas de interes facilita la compra de vivienda en 2024'}


In [13]:
# titulo + contenido
num_caracteres = [len(doc.page_content) for doc in texts]
describe_docs(num_caracteres)

min: 265 mean: 448 median: 449 max: 494


In [21]:
# save file
with open('chunks.bin', 'wb') as handle:
    pickle.dump(texts, handle)

: 

### Generacion de embedding de ejemplo

In [17]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('intfloat/multilingual-e5-small')

input_texts = [
    f'passage: {texts[0].page_content}',
]
embeddings = model.encode(input_texts, normalize_embeddings=True, show_progress_bar=True)

Batches: 100%|██████████| 1/1 [00:00<00:00,  2.48it/s]


In [18]:
print("Dimension del vector de embeddings", len(embeddings[0]))
embeddings[0][:10]

Dimension del vector de embeddings 384


array([-0.01406619,  0.00473391, -0.0263987 , -0.10681732,  0.03214642,
       -0.04709134,  0.03635797,  0.06220098,  0.09489433,  0.01204683],
      dtype=float32)

In [19]:
# producto punto: validar que el vector este normalizado
embeddings @ embeddings.T

array([[1.]], dtype=float32)

---